In [6]:
import pandas as pd
from pathlib import Path
import numpy as np

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [13]:
data_path=Path("C:/Users/asult/Downloads/opengeo_ready/ml_week1_project/data/heart.csv")
df = pd.read_csv(data_path)
TARGET_COL= 'target'
bins = [0,40,50,60,100]
labels=["<=40","41-50","51-60",">60"]
df["age_group"]=pd.cut(df["age"],bins=bins,labels=labels,right=True)
df_encoded = pd.get_dummies(df,columns=["age_group"],drop_first= True)
feature_cols = [col for col in df_encoded.columns if col != TARGET_COL]
X = df_encoded[feature_cols]
y = df_encoded[TARGET_COL]

X.shape, y.shape


((303, 16), (303,))

In [15]:
log_reg=LogisticRegression(max_iter=1000)
rf=RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42
)

In [16]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)


In [17]:
scores_log = cross_val_score(
    log_reg, X, y,
    cv=cv,
    scoring="accuracy"
)

print("Logistic Regression CV scores:", scores_log)
print("Mean accuracy:", scores_log.mean())
print("Std:", scores_log.std())


C:\Users\asult\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\asult\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://s

Logistic Regression CV scores: [0.90163934 0.83606557 0.75409836 0.81666667 0.85      ]
Mean accuracy: 0.8316939890710382
Std: 0.04794470871651464


C:\Users\asult\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
scores_rf=cross_val_score(
    rf,X,y,
    cv=cv,
    scoring="accuracy"
)
print("Random Forest CV scores",scores_rf)
print("Mean accuracy",scores_rf.mean())
print("Std:",scores_rf.std())

Random Forest CV scores [0.8852459  0.83606557 0.73770492 0.83333333 0.83333333]
Mean accuracy 0.825136612021858
Std: 0.04798175173445943


In [20]:
import pandas as pd

results_df = pd.DataFrame({
    "model": ["Logistic Regression", "Random Forest"],
    "mean_accuracy": [scores_log.mean(), scores_rf.mean()],
    "std_accuracy": [scores_log.std(), scores_rf.std()]
})

results_df

,model,mean_accuracy,std_accuracy
0,Logistic Regression,0.831694,0.047945
1,Random Forest,0.825137,0.047982


In [21]:
results_df.sort_values("mean_accuracy", ascending=False)


,model,mean_accuracy,std_accuracy
0,Logistic Regression,0.831694,0.047945
1,Random Forest,0.825137,0.047982


In [25]:
#if random forest is chosen
# Suppose Random Forest is chosen
final_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42
)

final_model.fit(X, y)


,n_estimators,200
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [26]:
import joblib
import os

os.makedirs("models", exist_ok=True)

model_path = "models/heart_rf_model.joblib"  # rename if you used log_reg
joblib.dump(final_model, model_path)

print("Model saved to:", model_path)


Model saved to: models/heart_rf_model.joblib


In [27]:
loaded_model = joblib.load(model_path)

# Take first 5 rows as a quick test
X_sample = X.iloc[:5]
y_sample = y.iloc[:5]

y_sample_pred = loaded_model.predict(X_sample)

print("True labels:     ", y_sample.to_list())
print("Predicted labels:", y_sample_pred.tolist())


True labels:      [1, 1, 1, 1, 1]
Predicted labels: [1, 1, 1, 1, 1]


In [30]:
def print_cv_results(name,scores):
    print(f"Model:{name}")
    print("Scores:", [f"{s:.3f}" for s in scores])
    print(f"Mean: {scores.mean():.3f}, Std: {scores.std():.3f}")
    print("-" * 40)


In [31]:
print_cv_results("Logistic Regression", scores_log)
print_cv_results("Random Forest", scores_rf)


Model:Logistic Regression
Scores: ['0.902', '0.836', '0.754', '0.817', '0.850']
Mean: 0.832, Std: 0.048
----------------------------------------
Model:Random Forest
Scores: ['0.885', '0.836', '0.738', '0.833', '0.833']
Mean: 0.825, Std: 0.048
----------------------------------------
